## Packages Import

In [4]:
import pandas as pd
import numpy as np
from functions import downloadData

ModuleNotFoundError: No module named 'functions'

## Data Download

In [ ]:

path = "C:\Users\Luke\Downloads\reviews_Electronics_5.json.gz"

data = 